# Python Web Access 的那些库

## 基础库：Socket

Socket是Python的用于HTTP通信的最基础包。

*以下程序基于Python for Everybody的学习课程。*

### Step 1：导入socket库

In [2]:
import socket

### Step 2：建立连接

socket库比较原始，所以cmd必须严格按照HTTP的命令格式书写，包括后面的'\r\n\r\n'书写不正确都会造成错误。此外最后需要用encode()命令将原本的unicode格式编码成binary发送。

In [41]:
mysock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
mysock.connect(('data.pr4e.org', 80))
cmd = 'GET http://data.pr4e.org/intro-short.txt HTTP/1.0\r\n\r\n'.encode()
mysock.send(cmd)

53

### Step 3：读取数据

接着我们就可以从建立的Socket来读取数据。用的方法是recv(buffersize)，buffersize是recv一次允许接收的最大数量内容。recv接收的内容为byte格式，所以我们在print输入的时候需要用decode()命令将它再次转化为unicode。

In [42]:
while True:
    data = mysock.recv(512)
    if len(data) < 1:
        break
    print(data.decode(),end='')

HTTP/1.1 200 OK
Date: Sun, 09 Oct 2022 12:56:52 GMT
Server: Apache/2.4.18 (Ubuntu)
Last-Modified: Sat, 13 May 2017 11:22:22 GMT
ETag: "1d3-54f6609240717"
Accept-Ranges: bytes
Content-Length: 467
Cache-Control: max-age=0, no-cache, no-store, must-revalidate
Pragma: no-cache
Expires: Wed, 11 Jan 1984 05:00:00 GMT
Connection: close
Content-Type: text/plain

Why should you learn to write programs?

Writing programs (or programming) is a very creative 
and rewarding activity.  You can write programs for 
many reasons, ranging from making your living to solving
a difficult data analysis problem to having fun to helping
someone else solve a problem.  This book assumes that 
everyone needs to know how to program, and that once 
you know how to program you will figure out what you want 
to do with your newfound skills.  


### Step 4：关闭socket

In [43]:
mysock.close()

### 总结

由于socket是非常低阶的HTTP连接方式，并且接收的数据不方便处理。所以在实际程序里我们很少用这个包。

## 进阶库1：urllib

urllib具体包含了四个组成部分：
1. urllib.request：用以打开和读取URL
2. urllib.error：包括了urllib.request产生的报错信息
3. urllib.parse：用以解析URL，包括添加parameter等操作
4. urllib.robotparser：用来解析robot.txt文件

详细内容够可以参考https://docs.python.org/3/library/urllib.html 

*以下程序很大程度上参考了Python for Everybody的课程内容*

### Step 1：导入基础库

In [143]:
import urllib.request, urllib.parse, urllib.error
import ssl
import json

这里我们没有导入整个urllib，因为robotparser那一部分我们不需要使用。

### Step 2：打开URL并获取数据

我们先来处理一下ssl加密证书。

In [144]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

接着就可以使用urlopen函数来打开URL了。

这里我们使用http://httpbin.org 这个API，它提供简单的HTTP request和response服务，并支持query parameters。

In [145]:
url_get='http://httpbin.org/get'
payload={"name":"Joseph","ID":"123"}

这里urllib库和后面提到的requests库的处理parameters的方式不太一样。
1. '?'需要手动添加
2. 使用函数urlencode可以将类型为dictionary的payload转化成query parameters

In [146]:
url = url_get + '?' + urllib.parse.urlencode(payload)
url

'http://httpbin.org/get?name=Joseph&ID=123'

接着我们就可以用urlopen来打开URL并从服务器端获得数据。用read()可以读取到类型为byte的数据。利用decode()可以将其转化为String类型。

在本例中，是否添加'context=ctx'的参数不影响输出结果。因为该API没有进行加密设置。

In [147]:
#r = urllib.request.urlopen(url)
r = urllib.request.urlopen(url, context=ctx)
data = r.read().decode()
print(type(r), type(data))
print(data)

<class 'http.client.HTTPResponse'> <class 'str'>
{
  "args": {
    "ID": "123", 
    "name": "Joseph"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.9", 
    "X-Amzn-Trace-Id": "Root=1-63432b00-25330f8b694ca54e71bf5833"
  }, 
  "origin": "87.189.151.85", 
  "url": "http://httpbin.org/get?name=Joseph&ID=123"
}



接着我们可以用用json来进一步将String类型的数据转换成dictionary，来方便进一步处理。

In [150]:
dict = json.loads(data)
print(dict.keys())

dict_keys(['args', 'headers', 'origin', 'url'])


### 总结

urllib是个相比于socket高阶很多的URL库，能实现更多的功能。

## 进阶库2：requests

requests是一个简单且非常受欢迎的python库，使用起来非常简单。通过和其他库的叠加可以实现网络数据的获取和分析。

*这部分内容基于IBM Skills提供的Python for Data Science, AI & Development。*

### Step 1：导入requests库

In [44]:
import requests

### Step 2：向web server发送request请求

简单的使用requests.get()就可以从指定的URL获得数据。我们将数据存储在response里，来仔细看一下response都包含一些什么内容。

In [59]:
url='https://www.ibm.com/'
response = requests.get(url)

requests库返回的response包含三部分：
1. response start line：其中包含了返回的数据。200表示ok，404表示page not found，除此之外还有很多其他状态。具体可以查询相关documentation。
2. response header：包含了网页的meta data，包括内容编码等具体信息。
3. response body：这里就是具体从网页上获取的数据了。

接下来我们具体看一下各个部分的内容。

In [60]:
print('Response start line:', response.status_code)

Response start line: 200


In [61]:
header = response.headers
print('Response header:')
print('Header type:', type(header))
print(header)

Response header:
Header type: <class 'requests.structures.CaseInsensitiveDict'>
{'Server': 'Apache', 'x-drupal-dynamic-cache': 'UNCACHEABLE', 'Link': '<https://www.ibm.com/de-de>; rel="canonical", <//1.cms.s81c.com>; rel=preconnect; crossorigin, <//1.cms.s81c.com>; rel=dns-prefetch', 'x-ua-compatible': 'IE=edge', 'Content-Language': 'de-de', 'Permissions-Policy': 'interest-cohort=()', 'x-generator': 'Drupal 9 (https://www.drupal.org)', 'x-dns-prefetch-control': 'on', 'x-drupal-cache': 'MISS', 'Last-Modified': 'Sun, 09 Oct 2022 11:49:49 GMT', 'ETag': '"1665316189"', 'Content-Type': 'text/html; charset=UTF-8', 'x-acquia-host': 'www.ibm.com', 'x-acquia-path': '/de-de', 'x-acquia-site': '', 'x-acquia-purge-tags': '', 'x-varnish': '783984292 789261231', 'x-cache-hits': '3', 'x-age': '4388', 'Accept-Ranges': 'bytes', 'Content-Encoding': 'gzip', 'Cache-Control': 'public, max-age=300', 'Expires': 'Sun, 09 Oct 2022 13:34:48 GMT', 'X-Akamai-Transformed': '9 10613 0 pmb=mTOE,2', 'Date': 'Sun, 09 

这里有一个比较烧脑的概念，我们上面看到的header是response的header部分。除了response，request其实也有三个部分组成，分别是：
1. request start line: 这里通常包含了GET指令，基本内容等同于上一节socket库里我们讲的cmd。举例：'GET http://data.pr4e.org/intro-short.txt HTTP/1.0'，request库会自动给指令转换编码。
2. request header：里面是request里的meta data
3. request body：request的body部分为空。

我们通过reponse可以看到request的header和body内容，具体如下：

In [66]:
print('Request header:')
print(response.request.headers)

Request header:
{'User-Agent': 'python-requests/2.27.1', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive', 'Cookie': '_abck=FA838D7453DFFF5BD1581CC820A2A2F4~-1~YAAQVIUVAmKY0oCDAQAAgf/vvAiDkfslAf2sUMcUZsl1IaWBiF+9GbNuIE0kDMkSbXE/E0F7xk9DHarF28ijimamT1IYzZpAYHXgPQ+Gy2L/bLF8bOMAw9B5SAO7uJ1tdKN6UQ8DkQWMRoV4OzIHp1W746A9yq19C6oMuVdCVMwWjW7gN76KeWQC+Py9RD5skQv9Pn+BmYFqLStvYXrreIWNKEZPvLX5IIcBqBL+hLvW8dji+Dt/rtyUQQ/QpPTqe0/mk/yLdBZrdQnOrgAwx0mg5ehkD84xpXwIdNwBpBQoN6DB98+az+fVftjLz5LMs14MBJSuV8lpkQQxarGgoOIeYIypMHAzYzLvHu9upwrjzPCE+38=~-1~-1~-1; bm_sz=3E568EA145059644EF1F9CF7EE947F35~YAAQVIUVAmOY0oCDAQAAgf/vvBERs6Y5WmPpHRHAGKqF8QR0Sj/bmTkuv40xi2UDsOsr1VEx6Cc+qzxV6BscfkVzrHOUQH+kaBPK4tn91yOlV1UEo6zgm9EFVqH/2DU1+KF6XwRftyRST1/yH1yAZ2lcaI0yMsto0lTh8VeQZbFK8nlfGLdm1/8j9ghS1fIJCBnqg83yjceBp2AX2Gg2ttu2mXMr3Cs8holp0vSeJCRbaFALgNM26gJ9fZZe/hpdWiVbyGfv/5j+uA+TZLEh2UbopvflxLgNh6fPHVCECCA=~3359809~4408375'}


**这里对比上面的response header我们可以发现内容是不一样的。**

In [67]:
print('Request body:')
print(response.request.body)

Request body:
None


### Step 3：分析获取的数据

接下来我们来仔细拆分一下response body，也就是我们从网站获取的数据。
首先我们调取一下reponse header**content-type**来看一下我们获取到的是什么数据。

In [68]:
header['content-type']

'text/html; charset=UTF-8'

这里我们可以看到网页返回的数据类型是html格式。requests库提供了content和text两个属性给我们调用内容。
- content：返回的是binary类型的原始数据
- text：返回的是经过解码的String类型的数据

In [72]:
print('Type of content:', type(response.content))
print('Type of text:', type(response.text))

Type of content: <class 'bytes'>
Type of text: <class 'str'>


此外我们用encoding属性可以看到网页数据的编码格式。

In [73]:
response.encoding

'UTF-8'

有的时候网站返回的不是html格式的内容。下面我们看一个例子。

http://httpbin.org 提供简单的HTTP request和response服务。我们直接从该网站获取一个json文件。具体如下：

In [77]:
url='http://httpbin.org/json'
r = requests.get(url)

In [83]:
print(r.status_code)
print(r.headers['content-type'])

200
application/json


从headers里我们可以看到，我们接收到的是一个json文件。requests库专门提供了一个解析json文件的函数json()，利用这个函数可以直接把获取的json文件转换成python的dictionary，具体如下：

In [92]:
data = r.json()
print(type(data))
print(data)

<class 'dict'>
{'slideshow': {'author': 'Yours Truly', 'date': 'date of publication', 'slides': [{'title': 'Wake up to WonderWidgets!', 'type': 'all'}, {'items': ['Why <em>WonderWidgets</em> are great', 'Who <em>buys</em> WonderWidgets'], 'title': 'Overview', 'type': 'all'}], 'title': 'Sample Slide Show'}}


这里我们发现data是个嵌套的dictionary，接下来我们可以用常用的方法来处理接收到的数据。甚至可以用pandas把数据转换成dataframe来进行处理。

In [112]:
import pandas as pd

In [113]:
true_data = data['slideshow']
print(true_data.keys())
df = pd.DataFrame(true_data)
df

dict_keys(['author', 'date', 'slides', 'title'])


,author,date,slides,title
0,Yours Truly,date of publication,"{'title': 'Wake up to WonderWidgets!', 'type':...",Sample Slide Show
1,Yours Truly,date of publication,{'items': ['Why <em>WonderWidgets</em> are gre...,Sample Slide Show


仔细看这里由于slides里是一个有两个element的List， 所以pandas自动把数据拆分成了两行。

HTTP request可以返回的数据类型除了上面提到的html和json以外还有很多种。具体处理方法我们需要具体分析。这部分内容会在其他文件里进一步分析。

### Step 4：给URL添加query parameters

除了上面我们看到的普通URL，有些URL是可以添加query parameters的，例如'http://httpbin.org/get?name=Joseph&ID=123' ，'?'代表后面跟的是parameters，这里我们看到有两个parameters，分别是'name=Jsseph'和'ID=123'。

In [114]:
import json

In [115]:
url_get='http://httpbin.org/get'
payload={"name":"Joseph","ID":"123"}
r=requests.get(url_get,params=payload)

我们可以用url属性来查看一下实际的url是什么。

In [116]:
r.url

'http://httpbin.org/get?name=Joseph&ID=123'

接着我们来查看一下获取数据的返回结果。

In [117]:
# get the status code
if r.status_code == 200:
    print('Data retrieved successfully.')
else:
    print('Something wrong happened.')

Data retrieved successfully.


In [118]:
# get the type of the content
print(r.headers['content-type'])

application/json


我们除了使用之前提到的r.json()来返回内容数据以外，还可以用json库的更为强大的函数来获取内容。
如下面那个例子：
1. 用text属性获得String类型的内容
2. 用json.load()将其转换为dictionary
3. 用json.dumps()来整理格式并输出

In [121]:
results = json.loads(r.text)
print(type(r), type(r.text), type(results))

<class 'requests.models.Response'> <class 'str'> <class 'dict'>


In [127]:
print(json.dumps(results, indent=4))

{
    "args": {
        "ID": "123",
        "name": "Joseph"
    },
    "headers": {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Host": "httpbin.org",
        "User-Agent": "python-requests/2.27.1",
        "X-Amzn-Trace-Id": "Root=1-63431d5f-061af70d60db137e3526f046"
    },
    "origin": "87.189.151.85",
    "url": "http://httpbin.org/get?name=Joseph&ID=123"
}


### 总结

requests库是一个功能非常强大的库，对于返回的response的控制有了更好的灵活。**是最值得推荐使用的一个库。**